In [1]:
from utils import *
import os
import shutil

In [2]:
ModelPath = r'E:\data\MODELS' 

OneStageModelName = 'onestage_model'
TwoStageModelName = 'twostage_model'

SfMPath = r'E:\data\testagain2'
OriginalImageFolder = r'E:\data\jpgimages'

SfMSparseModel = 'sparsemodel.nvm'
SfMTextureFolder = 'textures'
OutputFolder = 'superimposedresults'

OpenMVSPath = r'C:\Users\assembled\OpenMVS\src\bin\vc15\x64\Release'
VisualSFMPath = r'F:\cx\VisualSFM_windows_cuda_64bit'

In [3]:
dlab2,transform_deeplab = load_model(Number_of_Class = 8,
                                     Image_size = (900,900),
                                     Backbone = 'resnet',
                                     Path = ModelPath,
                                     Name = OneStageModelName)

dlab2_detail,transform_deeplab_detail = load_model(Number_of_Class = 2,
                                                   Image_size = (200,200),
                                                   Backbone = 'resnet',
                                                   Path = ModelPath,
                                                   Name = TwoStageModelName)

In [4]:
red = (255,0,0)
green = (0,255,0)
blue = (0,0,255)
yellow = (255,225,0)
orange = (255,125,0)
purple = (155,0,255)
greenblue = (0,255,225)
pink = (255,140,248)
white = (255,255,255)
black = (0,0,0)
colors = [red,green,blue,yellow,orange,purple,greenblue,pink]

In [5]:
if os.path.exists(SfMPath) == False:
    os.mkdir(SfMPath)
    
if os.path.exists(r'{0}\{1}'.format(SfMPath,SfMTextureFolder)) == False:
    os.mkdir(r'{0}\{1}'.format(SfMPath,SfMTextureFolder))

for i in os.listdir(OriginalImageFolder):
    shutil.copy(r'{0}\{1}'.format(OriginalImageFolder,i),r'{0}\{1}\{2}'.format(SfMPath,SfMTextureFolder,i))

os.chdir(VisualSFMPath)

os.popen(r'VisualSFM sfm {0}\{1} {0}\{2}'.format(SfMPath,SfMTextureFolder,SfMSparseModel.split('.')[0])).read()

print('finished Visual SFM')

making_masks(colors = colors,
             mask_alpha = 0.5,
             Path = SfMPath,
             InputFolder = OriginalImageFolder,
             OutputFolder = OutputFolder,
             transform_deeplab = transform_deeplab,
             transform_deeplab_detail = transform_deeplab_detail,
             dlab2 = dlab2,
             dlab2_detail = dlab2_detail,
             class_label = 'all',
             two_stage = False,
             scale = 0.05)

print('finished making masks')

os.chdir(OpenMVSPath)

os.popen(r'InterfaceVisualSFM -w {0} {1}'.format(SfMPath,SfMSparseModel)).read()

print('finished reading {0}'.format(SfMSparseModel))

os.popen(r'ReconstructMesh -w {0} {1}.mvs'.format(SfMPath,SfMSparseModel.split('.')[0])).read()

print('finished reconstructing mesh')

for i in os.listdir(r'{0}\{1}'.format(SfMPath,OutputFolder)):
    shutil.copy(r'{0}\{1}\{2}'.format(SfMPath,OutputFolder,i),r'{0}\{1}\{2}'.format(SfMPath,SfMTextureFolder,i))
    
os.popen(r'TextureMesh -w {0} {1}_mesh.mvs'.format(SfMPath,SfMSparseModel.split('.')[0])).read()

print('finished texturing')

finished Visual SFM
finished 0.jpg
finished 1.jpg
finished 10.jpg
finished 11.jpg
finished 12.jpg
finished 13.jpg
finished 14.jpg
finished 15.jpg
finished 16.jpg
finished 17.jpg
finished 18.jpg
finished 19.jpg
finished 2.jpg
finished 20.jpg
finished 21.jpg
finished 22.jpg
finished 23.jpg
finished 24.jpg
finished 25.jpg
finished 26.jpg
finished 27.jpg
finished 28.jpg
finished 29.jpg
finished 3.jpg
finished 30.jpg
finished 31.jpg
finished 32.jpg
finished 33.jpg
finished 34.jpg
finished 35.jpg
finished 36.jpg
finished 37.jpg
finished 38.jpg
finished 39.jpg
finished 4.jpg
finished 40.jpg
finished 41.jpg
finished 42.jpg
finished 43.jpg
finished 44.jpg
finished 45.jpg
finished 46.jpg
finished 47.jpg
finished 48.jpg
finished 49.jpg
finished 5.jpg
finished 50.jpg
finished 51.jpg
finished 52.jpg
finished 53.jpg
finished 54.jpg
finished 55.jpg
finished 56.jpg
finished 57.jpg
finished 58.jpg
finished 59.jpg
finished 6.jpg
finished 60.jpg
finished 61.jpg
finished 62.jpg
finished 63.jpg
finished 64